In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.regularizers import l2

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown


In [5]:
import zipfile

Zipref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
Zipref.extractall('/content')
Zipref.close()

In [6]:
train_data = tf.keras.utils.image_dataset_from_directory(
    directory='/content/train',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256, 256),
)

val_data = tf.keras.utils.image_dataset_from_directory(
    directory='/content/test',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256, 256),
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [7]:
def preprocess(image, label):
    image = tf.cast(image/255.0, tf.float32)
    return image, label

In [8]:
train_data = train_data.map(preprocess)
val_data = val_data.map(preprocess)

In [10]:
model = Sequential()

## Convolutional layers
model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=1, padding='valid', activation='relu', input_shape=(256, 256, 3), name='2d_conv_l1'))
model.add(BatchNormalization(name='batch_normalization_l1'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid', name='max_pooling_l1'))

model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=1, padding='valid', activation='relu', name='2d_conv_l2'))
model.add(BatchNormalization(name='batch_normalization_l2'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid', name='max_pooling_l2'))

model.add(Conv2D(filters=128, kernel_size=(3, 3), strides=1, padding='valid', activation='relu', name='2d_conv_l3'))
model.add(BatchNormalization(name='batch_normalization_l3'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid', name='max_pooling_l3'))

## Flatten Layer
model.add(Flatten(name='flatten'))

## Fully connected layers
model.add(Dense(units=128, activation='relu', kernel_regularizer=l2, name='fully_connected_layer_1'))
model.add(BatchNormalization(name='batch_normalization_l4'))
model.add(Dropout(0.1, name='Dropout_l1'))

model.add(Dense(units=64, activation='relu', kernel_regularizer=l2, name='fully_connected_layer_2'))
model.add(BatchNormalization(name='batch_normalization_l5'))
model.add(Dropout(0.1, name='Dropout_l2'))

model.add(Dense(units=1, activation='sigmoid', kernel_regularizer=l2, name='fully_connected_layer_3'))

model.name = 'cat_dog_cls'

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "cat_dog_cls"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ 2d_conv_l1 (Conv2D)             │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_l1          │ (None, 254, 254, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling_l1 (MaxPooling2D)   │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ 2d_conv_l2 (Conv2D)             │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_l2          │ (None, 125, 125, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling_l2 (MaxPooling2D)   │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ 2d_conv_l3 (Conv2D)             │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_l3          │ (None, 60, 60, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling_l3 (MaxPooling2D)   │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fully_connected_layer_1 (Dense) │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_l4          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout_l1 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fully_connected_layer_2 (Dense) │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_l5          │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout_l2 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fully_connected_layer_3 (Dense) │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,848,961 (56.64 MB)

 Trainable params: 14,848,129 (56.64 MB)

 Non-trainable params: 832 (3.25 KB)

In [11]:
history = model.fit(train_data, epochs=50, validation_data=val_data).history
model.save('cat_dog_cls.keras')

Epoch 1/100
625/625 ━━━━━━━━━━━━━━━━━━━━ 65s 83ms/step - accuracy: 0.6418 - loss: 2.9575 - val_accuracy: 0.6544 - val_loss: 1.3015
Epoch 2/100
625/625 ━━━━━━━━━━━━━━━━━━━━ 69s 76ms/step - accuracy: 0.7180 - loss: 1.2076 - val_accuracy: 0.7022 - val_loss: 1.2060
Epoch 3/100
625/625 ━━━━━━━━━━━━━━━━━━━━ 52s 83ms/step - accuracy: 0.7603 - loss: 1.1389 - val_accuracy: 0.5504 - val_loss: 2.0190
Epoch 4/100
625/625 ━━━━━━━━━━━━━━━━━━━━ 48s 76ms/step - accuracy: 0.7894 - loss: 1.3370 - val_accuracy: 0.7576 - val_loss: 1.4694
Epoch 5/100
625/625 ━━━━━━━━━━━━━━━━━━━━ 48s 77ms/step - accuracy: 0.8134 - loss: 1.4337 - val_accuracy: 0.7976 - val_loss: 2.3633
Epoch 6/100
625/625 ━━━━━━━━━━━━━━━━━━━━ 48s 77ms/step - accuracy: 0.8219 - loss: 1.7825 - val_accuracy: 0.7858 - val_loss: 1.5869
Epoch 7/100
625/625 ━━━━━━━━━━━━━━━━━━━━ 49s 78ms/step - accuracy: 0.8394 - loss: 1.6564 - val_accuracy: 0.7792 - val_loss: 1.8652
Epoch 8/100
625/625 ━━━━━━━━━━━━━━━━━━━━ 48s 77ms/step - accuracy: 0.8571 - loss: 1

KeyboardInterrupt: 

## Training
TRAINED FOR 45 EPOCHS / 100 EPOCHS

In [13]:
history = model.fit(train_data, epochs=50, validation_data=val_data, initial_epoch=45).history

Epoch 46/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 52s 83ms/step - accuracy: 0.9697 - loss: 0.3831 - val_accuracy: 0.8942 - val_loss: 0.6917
Epoch 47/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 78s 76ms/step - accuracy: 0.9642 - loss: 0.4544 - val_accuracy: 0.9002 - val_loss: 0.6304
Epoch 48/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 52s 83ms/step - accuracy: 0.9618 - loss: 0.4266 - val_accuracy: 0.8528 - val_loss: 0.7847
Epoch 49/50


KeyboardInterrupt: 

In [14]:
model.save('cat_dog_cls.keras')

In [ ]:
plt.plot(history['loss'], label='train loss', color='red')
plt.plot(history['val_loss'], label='val loss', color='green')
plt.title('Loss')
plt.legend()
plt.show()

plt.plot(history['accuracy'], label='train accuracy', color='red')
plt.plot(history['val_accuracy'], label='val accuracy', color='green')
plt.title('Accuracy')
plt.legend()
plt.show()

# Load the trained model locally

In [ ]:
trained_model = keras.models.load_model('cat_dog_cls.keras')
model.summary()

## Test Model on Unseen Data

In [ ]:

test_img = cv2.imread(test_img_path)
plt.imshow(test_img)
plt.show()

test_img.shape

In [ ]:
test_img = cv2.resize(test_img, (256, 256))
test_img = test_img.reshape(1, 256, 256, 3)

In [ ]:
def map_label_to_text(label):
    if label == 0:
        return 'Cat'
    else:
        return 'Dog'

In [ ]:
model.predict(test_img)